# DreamJobber

**Tech Edition**

---

**Process**
1. Clean text
2. Bag of Words
3. LDA model (Latent Dirichlet allocation)
4. Fine tune LDA model
5. Define Topics from LDA model
6. Create df of document probabilities
6. Classification model

---

**Import Necessary Libraries**

In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from functions import *
import pickle

#lda model evaluatoin with coherence
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

from sklearn.neighbors import NearestNeighbors

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---

**Load Data**

In [38]:
df_1 = pd.read_json('data/dice_jobs_1.json', lines=True)
df_2 = pd.read_json('data/dice_jobs_2.json', lines=True)
df_3 = pd.read_json('data/dice_jobs_3.json', lines=True)
df_4 = pd.read_json('data/dice_jobs_4.json', lines=True)
df_5 = pd.read_json('data/dice_jobs_5.json', lines=True)
df_6 = pd.read_json('data/dice_jobs_6.json', lines=True)

In [39]:
#concat into one df
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True, sort=True)

In [40]:
df.head()

,job_description,job_title
0,NaN,UI Lead/Architect
1,NaN,Web Application Architect
2,NaN,Senior DataStage Developer
3,NaN,Hadoop Administrator
4,NaN,UX Visual Designer


In [41]:
#check for missing values
df.isna().sum()

job_description    6524
job_title          5484
dtype: int64

In [42]:
#looks like there are rows that have no job description
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27016 entries, 0 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          21532 non-null object
dtypes: object(2)
memory usage: 422.2+ KB


In [43]:
#drop rows with no job descriptions
df = df.dropna()

In [44]:
#sanity check, looks good
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20492 entries, 9 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 480.3+ KB


In [45]:
df.head()

,job_description,job_title
9,[2+ years of experience developing Java / J2EE...,Java Developer (Sign-On BONUS!)
10,"[Passion for technology and learning, a natura...","iOS Developer - Mobile Rate: Open, Duration: 1..."
17,[We enjoy approved IT vendor status with sever...,"EUC Engineer, Rate-Open, Duration: 18 Months"
18,[We enjoy approved IT vendor status with sever...,"Software Developer - RPG, Rate-Open, Duration:..."
19,[SME in Linux Operating system with Strong Vir...,Sr. Linux Consultant with Weblogic exp


In [46]:
#need to remove brackets from job_description
df['job_description'] = df['job_description'].map(remove_brackets)

In [47]:
#remove '\\n' and replace with ','
df['job_description'] = df['job_description'].map(lambda x: x.replace('\\n', ','))

In [48]:
#lowercase text before applying stopwords
df['job_description'] = df['job_description'].map(lambda x: x.lower())

In [49]:
df.head()

,job_description,job_title
9,'2+ years of experience developing java / j2ee...,Java Developer (Sign-On BONUS!)
10,"'passion for technology and learning, a natura...","iOS Developer - Mobile Rate: Open, Duration: 1..."
17,'we enjoy approved it vendor status with sever...,"EUC Engineer, Rate-Open, Duration: 18 Months"
18,'we enjoy approved it vendor status with sever...,"Software Developer - RPG, Rate-Open, Duration:..."
19,'sme in linux operating system with strong vir...,Sr. Linux Consultant with Weblogic exp


In [50]:
#lowercase job_title text before cleaning
df['job_title'] = df['job_title'].map(lambda x: x.lower())

In [35]:
#def remove_stop_words(list1):
 #   result = []
  #  for word in list1:
   #     if word not in jobtitle_stopwords:
    #        result.append(word)
    #return result

In [36]:
#df['job_title'] = df['job_title'].apply(remove_stop_words)

In [51]:
df.head()

,job_description,job_title
9,'2+ years of experience developing java / j2ee...,java developer (sign-on bonus!)
10,"'passion for technology and learning, a natura...","ios developer - mobile rate: open, duration: 1..."
17,'we enjoy approved it vendor status with sever...,"euc engineer, rate-open, duration: 18 months"
18,'we enjoy approved it vendor status with sever...,"software developer - rpg, rate-open, duration:..."
19,'sme in linux operating system with strong vir...,sr. linux consultant with weblogic exp


---

## Text Cleaning

1. Tokenize
2. Remove words with fewer than 3 characters
3. Remove stop words
4. Normalize words (Lemmatize and Stem)

**Test the functions on one row of text**

In [15]:
stemmer = SnowballStemmer('english')

In [16]:
text_sample = df[df.index == 14677].values[0][0]

print('original text: ')
words = []
for word in text_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized text: ')
print(preprocess(text_sample))

original text: 
["'our", 'client,', 'a', 'top', 'national', 'optometry', 'retailer', 'is', 'currently', 'looking', 'for', 'a', 'hris', 'analyst', 'to', 'help', 'command', 'and', 'support', 'their', 'current', 'human', 'resource', 'system', 'functions.', 'our', 'client', 'heavily', 'works', 'in', 'a', 'ceridian', 'dayforce', 'environment', 'is', 'looking', 'for', 'an', 'experience', 'resource', 'to', 'support', 'their', 'hr', 'team.,role', "summary:',", "'the", 'human', 'resources', 'information', 'systems', '(hris)', 'analyst', 'is', 'part', 'of', 'the', 'overall', 'hr', 'team', 'and', 'is', 'a', 'key', 'partner', 'and', 'liaison', 'to', 'all', 'centers', 'of', 'excellence.', 'the', 'hris', 'analyst', 'is', 'not', 'only', 'responsible', 'for', 'comprehensive', 'and', 'strategic', 'hr', 'systems', 'administration,', 'but', 'also', 'acts', 'as', 'a', 'business', 'partner', 'to', 'other', 'members', 'of', 'the', 'team,', 'ensuring', 'that', 'all', 'hr', 'centers', 'of', 'excellence', 'hav

**Apply functions to job_description**

In [17]:
#apply function and display first 5 rows
processed_text = df['job_description'].map(preprocess)
processed_text[:5]

9                         [year, develop, java, applic]
10    [passion, technolog, learn, natur, curios, lov...
17    [enjoy, approv, vendor, status, lead, compani,...
18    [enjoy, approv, vendor, status, lead, compani,...
19    [linux, oper, strong, virtual, knowledg, vmwar...
Name: job_description, dtype: object

---

## Bag of Words

In [18]:
#I'll use bag of words to extract features from text for use in modeling

In [19]:
dictionary = gensim.corpora.Dictionary(processed_text)

In [20]:
#check the length before I filter out the extremes
len(dictionary)

21777

In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [22]:
#check length after filtering out extremes
len(dictionary)

4445

In [23]:
#bow2doc: counts the number of occurrences of each distinct word, 
#converts the word to its integer word id and returns the result as a sparse vector

bow2doc_corpus = [dictionary.doc2bow(text) for text in processed_text]

In [ ]:
##let's take a look
#bow_doc_5000 = bow2doc_corpus[5000]

#for i in range(len(bow_doc_5000)):
 #   print("Word {} (\"{}\") appears {} time.".format(bow_doc_5000[i][0], 
  #                                                   dictionary[bow_doc_5000[i][0]], 
   #                                                  bow_doc_5000[i][1]))

---

## Find optimal number of topics

In [ ]:
#model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow2doc_corpus,
 #                                                       texts=processed_text, start=5, limit=40, step=5)

In [ ]:
#import matplotlib.pyplot as plt
#limit=40; start=5; step=5;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Number of Topics")
#plt.ylabel("Coherence score")
#plt.show()

---

## LDA model with Bag of Words

In [24]:
lda_model = gensim.models.LdaMulticore(bow2doc_corpus, 
                                       num_topics=9, 
                                       id2word=dictionary, 
                                       passes=100, 
                                       workers=4,
                                      chunksize=200)


In [25]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.036*"project" + 0.028*"busi" + 0.014*"process" + 0.013*"abil" + 0.012*"technic" + 0.011*"requir" + 0.011*"plan" + 0.010*"solut" + 0.010*"product" + 0.009*"provid"
Topic: 1 
Words: 0.023*"compani" + 0.015*"posit" + 0.015*"look" + 0.013*"solut" + 0.012*"client" + 0.012*"help" + 0.010*"industri" + 0.009*"need" + 0.009*"world" + 0.008*"grow"
Topic: 2 
Words: 0.048*"secur" + 0.034*"network" + 0.021*"system" + 0.014*"administr" + 0.014*"oper" + 0.014*"server" + 0.014*"infrastructur" + 0.014*"configur" + 0.012*"engin" + 0.010*"implement"
Topic: 3 
Words: 0.067*"test" + 0.019*"user" + 0.017*"softwar" + 0.016*"autom" + 0.015*"requir" + 0.014*"applic" + 0.013*"integr" + 0.012*"document" + 0.011*"function" + 0.010*"salesforc"
Topic: 4 
Words: 0.021*"status" + 0.018*"applic" + 0.018*"protect" + 0.017*"receiv" + 0.014*"origin" + 0.013*"qualifi" + 0.013*"ident" + 0.013*"posit" + 0.012*"elig" + 0.012*"busi"
Topic: 5 
Words: 0.026*"softwar" + 0.022*"java" + 0.021*"engin" + 0.015*"cl

**Pickled LDA model results**

In [ ]:
#pickle.dump(lda_model, open('pickled_models/lda_model.pkl', 'wb'))
pickled_lda = pickle.load(open('pickled_models/lda_model.pkl', 'rb'))

In [ ]:
for idx, topic in pickled_lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

---

**Coherence Score**

In [ ]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Compute Coherence Score using UMass
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#!pip install pyLDAvis

In [ ]:
##visualize the topics in order to better label 
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=bow2doc_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

---

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=bow2doc_corpus, texts=df['job_description'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


**Create df for topic scores for each jobtitle**

In [ ]:
topic_vecs = []
for i in range(len(bow2doc_corpus)):
    top_topics = pickled_lda.get_document_topics(bow2doc_corpus[i], minimum_probability=0.0)
    #i in range(amount of topics)
    topic_vec = [top_topics[i][1] for i in range(9)]
    topic_vecs.append(topic_vec)

In [ ]:
df_topic_vecs = pd.DataFrame(topic_vecs)
df_topic_vecs.head(10)

In [ ]:
#name columns for df
col_names=['Database Admin', 'Computer Network', 'Software/App Dev', 'Security', 
           'Analyst', 'Leadership', 'Computer Support', 
           'WebDev', 'Cloud Computing']
df_topic_vecs.columns = col_names
df_topic_vecs.head()

---

---

# Nearest Neighbors

In [ ]:
#next step merge with original df of job titles and job descriptions
#pickle the merged df

In [ ]:
df_final = pd.merge(df, df_topic_vecs,left_index=True, right_index=True)

In [ ]:
#pickle.dump(df_final, open('pickled_models/df_final.pkl', 'wb'))
pickled_df_final = pickle.load(open('pickled_models/df_final.pkl', 'rb'))
pickled_df_final.head()

In [ ]:
topics = pickled_df_final.drop(['job_description', 'job_title'], axis=1)
jobs = pickled_df_final['job_title']

In [ ]:
nearest_neighbor = NearestNeighbors(n_neighbors=50, metric='cosine')
nearest_neighbor.fit(topics)

**Make predictions**

In [ ]:
collect_score_and_recommend(nearest_neighbor, jobs)

In [ ]:
#next-steps
#1.more stopwords for descriptions(countries, cities, company names)
#2.find best topic separation and pickle
#3.stop words for job_titles
#4.find a way a to group similar job titles
#5.recommendations return unique list of top 10
